In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import operator
import gc

In [2]:
PATH = '../data/'

PROP_DATA_PATH = PATH + 'Property dataset/'
TRN_DATA_PATH = PATH + 'training dataset/'
TEST_DATA_PATH = PATH + 'Test dataset/'

##### Accounts Profiles

This table has information on customers/accounts.   
These are the accounts of whom we are marketing the properties for sale.

###### Profiles are created for every account(or customer) on the basis of following continuous features:
   * active_deals
   * active_deals
   * num_deals_as_client
   * num_deals_as_investor
   * number_of_properties
   * number_of_related_deals
   * number_of_related_properties
   * number_of_won_deals_as_client

###### and, following categorical features:
   * buyer_book
   * servicing_contract
   * cmbs
   * consultant
   * correspondent
   * foreign
   * master_servicer
   * lender_book
   * loan_sales_book
   * loan_servicing

Above features help in finding similar users to the accounts the we need to reccommend properties to.

In [3]:
accounts_table = pd.read_csv(TRN_DATA_PATH+'Accounts.csv')

accounts_table.head()

accounts_table.describe()

accts_table_cnts_features = [
                                'active_deals',
                                'activity_count',
                                'num_deals_as_client',
                                'num_deals_as_investor',
                                'number_of_properties',
                                'number_of_related_deals',
                                'number_of_related_properties',
                                'number_of_won_deals_as_client'
                            ]

accts_table_bool_vars = [
                            'buyer_book',
                            'servicing_contract',
                            'cmbs',
                            'consultant',
                            'correspondent',
                            'foreign',
                            'master_servicer',
                            'lender_book',
                            'loan_sales_book',
                            'loan_servicing'
                        ]

for var in accts_table_bool_vars:
    accounts_table[var].replace([True,False],[1,0],inplace=True)

display(accounts_table.head())

accts_ids = accounts_table.id_accs.values.tolist()

accts_profiles = accounts_table[accts_table_cnts_features+accts_table_bool_vars].values.astype('float')

,id_accs,active_deals,activity_count,buyer_book,servicing_contract,investor_type,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,loan_sales_book,loan_servicing,num_deals_as_client,num_deals_as_investor,number_of_properties,number_of_related_deals,number_of_related_properties,number_of_won_deals_as_client
0,0012A000023XlCCQA0,1,2367,1,0,Private Investor/Developer,0,0,0,0,0,0,0,0,0,1,1,2,1,0
1,0012A000023XmOxQAK,9,7974,1,0,REIT,0,0,0,0,0,1,1,0,7,2,77,18,85,7
2,0012A000023YLqwQAG,891,50161,1,1,Bank / Thrift,0,0,0,0,0,1,1,0,40,851,693,996,1167,37
3,0012A000023XlNjQAK,34,14631,1,0,Conduit/CMBS,1,0,0,1,0,1,1,0,0,34,0,45,14,0
4,0012A000023YBFRQA4,283,194195,1,0,Life Insurance Company,0,0,0,0,0,1,1,0,63,220,393,383,591,46


##### Properties profiles

This database contains the universal list of properties and its details

In [4]:
property_data = pd.read_csv(PROP_DATA_PATH+'Properties.csv')
property_data.drop('id_deals',axis=1,inplace=True)

In [5]:
display(property_data.head())

,id_props,building_status,building_tax_expenses,city,class,county,sale_status,portfolio,market,num_buildings,...,price_per_sq_ft,price_per_unit,property_type_1,region__c,sale_amount__c,sale_date__c,size_acres,size_sf,size_units,year_built
0,a0I2A00000XQGKlUAP,Existing,2009 Tax @ $1.25/sf,City-0488,Class B,County-0172,N,False,Market-0059,1.0,...,46.59,18350000.00,Industrial,Southwest,1.835000e+07,7/10/2015,3.760,393862.0,1.0,1980.0
1,a0I2A00000XQGKnUAP,NaN,NaN,City-1975,Class A,County-0465,Y,True,Market-0239,NaN,...,1141.55,2109090.91,Multi-Housing,Southeast,2.144058e+07,12/31/2016,19.600,508080.0,275.0,2014.0
2,a0I2A00000XQGKoUAP,Existing,"2016 Tax @ $5.63/sf; 2007 Ops @ $19.58/sf, 201...",City-1415,Class B,County-0452,N,False,Market-0165,1.0,...,1506.65,NaN,Retail,Northeast,2.754156e+07,9/30/2016,0.266,18280.0,NaN,1902.0
3,a0I2A00000XQGKpUAP,NaN,NaN,City-1415,Class A,County-0452,N,False,Market-0165,1.0,...,580.83,1220689.66,NaN,Northeast,1.416000e+08,1/21/2016,0.530,243789.0,116.0,NaN
4,a0I2A00000XQGKqUAP,NaN,NaN,City-0941,NaN,County-0284,N,False,Market-0106,NaN,...,NaN,NaN,Self-Storage,Southwest,0.000000e+00,9/30/2014,NaN,NaN,NaN,NaN


##### Looking at past successful deals

In [6]:
opportunities_table = pd.read_csv(TRN_DATA_PATH+'Opportunities.csv')
deal_to_properties_table = pd.read_csv(TRN_DATA_PATH+'Deals_to_Properties.csv')

In [7]:
display(opportunities_table.head())

,id_deals,id_accs,accounting_date,best_initial_bid,id_deals.1,closedate,date_closed,deal_type,deal_update_flag,debt_yield,fiscal,fiscalquarter,fiscalyear,platform,property_group,property_type
0,0062A00000moF1LQAU,0012A000023XvIOQA0,NaN,NaN,0062A00000moF1LQAU,8/12/2005,8/18/2005 0:00,Fixed-rate,True,NaN,2005 3,3,2005,Debt,Healthcare,Healthcare
1,0062A00000moF9LQAU,0012A000023YFe1QAG,NaN,NaN,0062A00000moF9LQAU,2/6/2012,6/27/2012 12:00,Fixed-rate,True,NaN,2012 1,1,2012,Debt,Healthcare,Biotech
2,0062A00000moGodQAE,0012A000023YEDcQAO,8/12/2004,NaN,0062A00000moGodQAE,11/24/2003,8/11/2004 0:00,Construction Loan,True,NaN,2003 4,4,2003,Debt,Healthcare,Biotech
3,0062A00000moBTHQA2,0012A000023YA3nQAG,NaN,NaN,0062A00000moBTHQA2,2/10/2006,4/19/2006 0:00,Fixed-rate,True,NaN,2006 1,1,2006,Debt,Healthcare,Healthcare
4,0062A00000mo8OjQAI,0012A000023XpRUQA0,NaN,NaN,0062A00000mo8OjQAI,7/13/2006,11/16/2006 0:00,Fixed-rate,True,NaN,2006 3,3,2006,Debt,Healthcare,Healthcare


In [8]:
display(deal_to_properties_table.head())

,id,createdbyid,id_deals,id_props
0,a0A2A00000TxIrIUAV,005F0000007WnNDIA0,0062A00000o979GQAQ,a0I2A00000XQH8YUAX
1,a0A2A00000TxIrSUAV,005F0000007WnNDIA0,0062A00000o94JrQAI,a0I2A00000XQJ4dUAH
2,a0A2A00000TxIrXUAV,005F0000007WnNDIA0,0062A00000o94GJQAY,a0I2A00000XQJL0UAP
3,a0A2A00000TxIrcUAF,005F0000007WnNDIA0,0062A00000o982fQAA,a0I2A00000XQKgkUAH
4,a0A2A00000TxIrhUAF,005F0000007WnNDIA0,0062A00000o96M4QAI,a0I2A00000XQMEhUAP


Merging opportunities table data with deal to properties in order to find sold and unsold properties.

In [9]:
past_deals = pd.merge(opportunities_table,deal_to_properties_table,on=['id_deals'],how='left')

In [10]:
display(past_deals.head())

,id_deals,id_accs,accounting_date,best_initial_bid,id_deals.1,closedate,date_closed,deal_type,deal_update_flag,debt_yield,fiscal,fiscalquarter,fiscalyear,platform,property_group,property_type,id,createdbyid,id_props
0,0062A00000moF1LQAU,0012A000023XvIOQA0,NaN,NaN,0062A00000moF1LQAU,8/12/2005,8/18/2005 0:00,Fixed-rate,True,NaN,2005 3,3,2005,Debt,Healthcare,Healthcare,a0A2A00000Tzm0nUAB,005F0000007WnNDIA0,a0I2A00000XQHWJUA5
1,0062A00000moF9LQAU,0012A000023YFe1QAG,NaN,NaN,0062A00000moF9LQAU,2/6/2012,6/27/2012 12:00,Fixed-rate,True,NaN,2012 1,1,2012,Debt,Healthcare,Biotech,a0A2A00000TzlSEUAZ,005F0000007WnNDIA0,a0I2A00000XQH1TUAX
2,0062A00000moGodQAE,0012A000023YEDcQAO,8/12/2004,NaN,0062A00000moGodQAE,11/24/2003,8/11/2004 0:00,Construction Loan,True,NaN,2003 4,4,2003,Debt,Healthcare,Biotech,NaN,NaN,NaN
3,0062A00000moBTHQA2,0012A000023YA3nQAG,NaN,NaN,0062A00000moBTHQA2,2/10/2006,4/19/2006 0:00,Fixed-rate,True,NaN,2006 1,1,2006,Debt,Healthcare,Healthcare,a0A2A00000TzorFUAR,005F0000007WnNDIA0,a0I2A00000XQK2nUAH
4,0062A00000mo8OjQAI,0012A000023XpRUQA0,NaN,NaN,0062A00000mo8OjQAI,7/13/2006,11/16/2006 0:00,Fixed-rate,True,NaN,2006 3,3,2006,Debt,Healthcare,Healthcare,a0A2A00000TzrLTUAZ,005F0000007WnNDIA0,a0I2A00000XQMA8UAP


In [11]:
past_deals.shape

(29362, 19)

Looking at all the previously bought properties by accounts

In [12]:
accounts_to_properties_table = pd.read_csv(TRN_DATA_PATH+'Accounts_properties.csv')
accounts_to_properties_table.drop('id_deals',axis=1,inplace=True)

In [13]:
s1 = set(accounts_to_properties_table.id_props.values.tolist())

In [14]:
s2 = set(past_deals.id_props.values.tolist())

In [15]:
sold_out_properties = list(s1.intersection(s2))

In [16]:
unsold_properties = property_data.loc[~property_data.id_props.isin(sold_out_properties)]

In [17]:
unsold_properties.shape

(6457, 23)

##### Using unsold properties data, we create individual property profiles with the help of some of the uncorrelated features.

Some of the numeric features used are:
* price_per_sq_ft
* num_buildings
* num_floors
* num_parking_spaces
* sale_amount__c
* size_sf
* year_built   
while, dropping some of the other parameters like size and price related having obvious correlations with above.

In [18]:
property_data_cnt_feats = [
                            'price_per_sq_ft',
                            'num_buildings',
                            'num_floors',
                            'num_parking_spaces',
                            'sale_amount__c',
                            'size_sf',
                            'year_built'
                        ]

# features_to_impute = [
#                         'price_per_sq_ft',
#                         'num_buildings',
#                         'num_floors',
#                         'num_parking_spaces',
#                         'sale_amount__c',
#                         'size_sf',
#                         'year_built'
#                      ]

# for feat in property_data_cnt_feats:
#     temp = property_data.groupby(['city'])[feat].agg('mean').reset_index()
#     cities = temp['city'].values.tolist()
#     feat_means = temp[feat].values.tolist()
#     for ct in cities:
#         property_data.loc[property_data['city']==ct,feat] = temp.loc[temp['city']==ct][feat].values[0]

property_ids = unsold_properties.id_props.values.tolist()

property_profiles = unsold_properties[property_data_cnt_feats].fillna(0).values

In [19]:
property_profiles.shape

(6457, 7)

###### Function to get Most Similar accounts given an account-id

Makes uses of cosine distance between their feature vectors

In [20]:
def get_similar_accounts(acct_id):
    acct_id_idx = accts_ids.index(acct_id)
    acct_id_feature_vector = accts_profiles[acct_id_idx].astype('float')
    reccs = [(i,scipy.spatial.distance.cosine(acct_id_feature_vector,apf.astype('float')))\
             for i,apf in enumerate(accts_profiles)]
    reccs.sort(key=operator.itemgetter(1))
    return reccs

##### Function to get top-k similar accounts on the basis of percentile.

In [21]:
def topk_accounts(accts_tuples_list):
    uniq = {}
    x = np.array([each[1] for each in accts_tuples_list])
    p = np.percentile(x,1)
    print("accounts_percentile",p)
    for tl in accts_tuples_list:
        if tl[1] not in uniq:
            uniq[tl[1]] = tl[0]     
        if tl[1] > p:
            break
    return uniq

##### Accounts profiling on the basis of their past successful Property deals

Aggregating property features for the accounts on the basis of previously bought properties.  
This gives rough idea about account feature vectors in property space.

In [22]:
d1 = pd.merge(accounts_to_properties_table,property_data,on=['id_props'],how='left')

# Define all the groupby transformations
GROUPBY_AGGREGATIONS = [
        {'groupby': ['id_accs'], 'select': 'price_per_sq_ft', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'num_buildings', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'num_floors', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'num_parking_spaces', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'sale_amount__c', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'size_sf', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'year_built', 'agg': 'mean'},
    ]

# Apply all the groupby transformations
for spec in GROUPBY_AGGREGATIONS:
    # Name of the aggregation we're applying
    agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']
    
    # Name of new feature
    new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])

    print("Grouping by {}, and aggregating {} with {}".format(
        spec['groupby'], spec['select'], agg_name
    ))
    # Unique list of features to select
    all_features = list(set(spec['groupby'] + [spec['select']]))
    # Perform the groupby
    gp = d1[all_features]. \
        groupby(spec['groupby'])[spec['select']]. \
        agg(spec['agg']). \
        reset_index(). \
        rename(index=str, columns={spec['select']: new_feature})

    d1 = d1.merge(gp, on=spec['groupby'], how='left')
     # Clear memory
    del gp
    gc.collect()

d11 = d1[
            [
                'id_accs',
                'id_accs_mean_price_per_sq_ft',
                'id_accs_mean_num_buildings',
                'id_accs_mean_num_floors',
                'id_accs_mean_num_parking_spaces',
                'id_accs_mean_sale_amount__c',
                'id_accs_mean_size_sf',
                'id_accs_mean_year_built'
            ]
        ]

d11_dash = d11.drop_duplicates()

acc_profiles_wrt_property_features = d11_dash.fillna(0).values[:,1:]

acc_profiles_wrt_property_features.shape

Grouping by ['id_accs'], and aggregating price_per_sq_ft with mean
Grouping by ['id_accs'], and aggregating num_buildings with mean
Grouping by ['id_accs'], and aggregating num_floors with mean
Grouping by ['id_accs'], and aggregating num_parking_spaces with mean
Grouping by ['id_accs'], and aggregating sale_amount__c with mean
Grouping by ['id_accs'], and aggregating size_sf with mean
Grouping by ['id_accs'], and aggregating year_built with mean


(2727, 7)

##### Generating Reccomendations

1. Given an account-id.  
2. First find top-k accounts similar to the given account-id.  
3. For every similar account, getting the acct profile vector in the property feature space.  
4. Based on the cosine distance between above vector and property vectors, most similar properties are     recommended.

In [23]:
test_data = pd.read_csv(TEST_DATA_PATH+'Test_Data.csv')
ids = list(test_data['id_accs'].values)

In [24]:
results = {}

for i,_id in enumerate(ids):
    most_similar_accts = get_similar_accounts(_id)
    print(len(most_similar_accts))
    filtered_most_similar_accts = topk_accounts(most_similar_accts)
    filtered_most_similar_accts = filtered_most_similar_accts.values()
#     filtered_most_similar_accts = filter_similar_accts_updated(most_similar_accts)
    print(len(filtered_most_similar_accts))
    print(filtered_most_similar_accts)
    top_reccs = []
    for most_similar_acct in filtered_most_similar_accts:
        if most_similar_acct > 2727:
            continue
        v1 = None
        v1 = acc_profiles_wrt_property_features[most_similar_acct].astype('float')
        reccs = []
        reccs = [(i,scipy.spatial.distance.cosine(v1,pf))\
                 if not np.isnan(scipy.spatial.distance.cosine(v1,pf)) else (i,0)
                 for i,pf in enumerate(property_profiles)]
        reccs.sort(key=operator.itemgetter(1))
        reccs = [r for r in reccs if r[1]>0]
        top_reccs.extend(set(reccs))
    uniq_top_reccs = {}
    utr_list = []
    for tr in top_reccs:
        if tr[0] not in uniq_top_reccs:
            uniq_top_reccs[tr[0]] = 1
            utr_list.append(tr)
    utr_list.sort(key=operator.itemgetter(1))
    print(len(utr_list))
    x = np.array([each[1] for each in utr_list])
#     if x.size==0:
#         continue
    p = np.percentile(x,1)
    print("property_percentile ",p)
    final_reccs = [property_ids[utrecc[0]] for utrecc in utr_list if utrecc[1]<=p]
    results[_id] = final_reccs
    print(len(results[_id]))
#     break

2756
accounts_percentile 1.330248456610228e-07
29
dict_values([79, 367, 1061, 426, 221, 1911, 2125, 1297, 1705, 1102, 1438, 1278, 29, 176, 1615, 315, 1703, 451, 586, 1032, 1371, 1468, 547, 755, 127, 140, 2153, 1612, 797])


/home/chandrashekhar/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


5971
property_percentile  1.7685478226336927e-08
60
2756
accounts_percentile 1.0244015202776069e-06
29
dict_values([6, 854, 1961, 22, 98, 470, 1395, 2729, 2136, 354, 2339, 227, 83, 2349, 24, 2034, 162, 122, 2132, 1203, 2139, 2555, 1680, 61, 2052, 479, 85, 287, 1164])
5971
property_percentile  3.073442061962695e-08
60
2756
accounts_percentile 9.680130415135047e-07
29
dict_values([22, 24, 85, 54, 6, 229, 479, 20, 227, 98, 2729, 924, 162, 1164, 854, 2164, 1961, 46, 2692, 1680, 1171, 333, 2311, 2555, 133, 683, 2132, 145, 1160])
5971
property_percentile  2.3059209275366754e-08
60
2756
accounts_percentile 7.712776423252076e-07
29
dict_values([200, 393, 382, 1842, 775, 237, 244, 2237, 15, 175, 290, 443, 203, 1800, 1156, 5, 1641, 2133, 1924, 1576, 90, 359, 649, 1784, 2216, 432, 654, 774, 1295])
5971
property_percentile  1.1274855238330162e-08
60
2756
accounts_percentile 0.0001269809498002761
29
dict_values([266, 820, 741, 727, 1508, 1309, 2350, 248, 1209, 846, 213, 730, 1109, 860, 437, 1956, 1

In [25]:
import csv
with open('subm_final.csv','w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['id_accs','id_prop'])
    for resid,res in results.items():
        for r in res:
            row = [resid]+[r]
            writer.writerow(row)